In [3]:
import duckdb

In [4]:
con = duckdb.connect("vanguard.db")

The source csv has the funds as columns - retrieve all these columns then format into some pivot sql

In [3]:
columns = con.execute("select * from (describe select * from performance_tmp2.csv) x where column_name != 'Date'").df()['column_name']
formatted_columns = ["cast("+c+" as double)" for c in columns]
joined_columns = ", ".join(formatted_columns)
joined_columns

'cast(LifeStrategy®20EquityFund as double), cast(LifeStrategy®40EquityFund as double), cast(LifeStrategy®60EquityFund as double), cast(LifeStrategy®80EquityFund as double), cast(LifeStrategy®100EquityFund as double), cast(SustainableLife4050EquityFund as double), cast(SustainableLife6070EquityFund as double), cast(SustainableLife8090EquityFund as double), cast(TargetRetirement2015Fund as double), cast(TargetRetirement2020Fund as double), cast(TargetRetirement2025Fund as double), cast(TargetRetirement2030Fund as double), cast(TargetRetirement2035Fund as double), cast(TargetRetirement2040Fund as double), cast(TargetRetirement2045Fund as double), cast(TargetRetirement2050Fund as double), cast(TargetRetirement2055Fund as double), cast(TargetRetirement2060Fund as double), cast(TargetRetirement2065Fund as double), cast(ActiveUKEquityFund as double), cast(EmergingMarketsBondFund as double), cast(EmergingMarketsStockIndexFund as double), cast(ESGDevelopedAsiaPacificAllCapUCITSETF as double), c

In [4]:
pivot_sql = "create table FundPerformance as select Date, strptime(date, '%b%Y') AsDate, Fund, Perf, Perf/100+1 as 'Decimal_Perf' from (unpivot performance_tmp2.csv on " + joined_columns + " into name Fund value Perf) x"
pivot_sql

"create table FundPerformance as select Date, strptime(date, '%b%Y') AsDate, Fund, Perf, Perf/100+1 as 'Decimal_Perf' from (unpivot performance_tmp2.csv on cast(LifeStrategy®20EquityFund as double), cast(LifeStrategy®40EquityFund as double), cast(LifeStrategy®60EquityFund as double), cast(LifeStrategy®80EquityFund as double), cast(LifeStrategy®100EquityFund as double), cast(SustainableLife4050EquityFund as double), cast(SustainableLife6070EquityFund as double), cast(SustainableLife8090EquityFund as double), cast(TargetRetirement2015Fund as double), cast(TargetRetirement2020Fund as double), cast(TargetRetirement2025Fund as double), cast(TargetRetirement2030Fund as double), cast(TargetRetirement2035Fund as double), cast(TargetRetirement2040Fund as double), cast(TargetRetirement2045Fund as double), cast(TargetRetirement2050Fund as double), cast(TargetRetirement2055Fund as double), cast(TargetRetirement2060Fund as double), cast(TargetRetirement2065Fund as double), cast(ActiveUKEquityFund a

In [5]:
con.execute(pivot_sql).df()

,Count
0,7846


In [6]:
con.execute("select AsDate, Perf, Decimal_Perf, (EXP(SUM(LN(Decimal_Perf)) OVER (ORDER BY AsDate))-1)*100 AS cum from FundPerformance where fund = 'LifeStrategy®60EquityFund' and AsDate between '2022-01-01' and '2022-12-31' order by AsDate asc").df(),

(       AsDate  Perf  Decimal_Perf        cum
 0  2022-01-01 -2.97        0.9703  -2.970000
 1  2022-02-01 -1.78        0.9822  -4.697134
 2  2022-03-01  1.17        1.0117  -3.582090
 3  2022-04-01 -3.06        0.9694  -6.532478
 4  2022-05-01 -0.62        0.9938  -7.111977
 5  2022-06-01 -4.13        0.9587 -10.948252
 6  2022-07-01  5.03        1.0503  -6.468950
 7  2022-08-01 -1.74        0.9826  -8.096390
 8  2022-09-01 -5.63        0.9437 -13.270563
 9  2022-10-01  1.86        1.0186 -11.657396
 10 2022-11-01  4.04        1.0404  -8.088354
 11 2022-12-01 -3.40        0.9660 -11.213350,)

In [7]:
con.execute("create table as select * from info_clean.csv").df()

,FundName,Region,RiskLevel,ManagementType,NumberofAssets,OngoingCharge
0,LifeStrategy®20EquityFund,Global,4,Index,24977,022
1,LifeStrategy®40EquityFund,Global,4,Index,29723,022
2,LifeStrategy®60EquityFund,Global,4,Index,28343,022
3,LifeStrategy®80EquityFund,Global,5,Index,23444,022
4,LifeStrategy®100EquityFund,Global,5,Index,7405,022
...,...,...,...,...,...,...
81,USInvestmentGradeCreditIndexFund,USA,4,Index,7035,012
82,USDCorporate13YearBondUCITSETF,Global,3,Index,1946,009
83,USDCorporateBondUCITSETF,Global,4,Index,8523,009
84,USDEmergingMarketsGovernmentBondUCITSETF,Emerging,4,Index,1291,025


In [7]:
con.close()